<a href="https://colab.research.google.com/github/caiowmp/UFBA/blob/master/C%C3%B3pia_de_P03_Classificando_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Classificando Tweets

# 1 - Defina o termo que você deseja buscar no Twitter. Pense em algo interessante em termos de classificação de uma postagem. 

Por exemplo, você pode procurar por Twittes que contenham palavras com possemia, tal como banco, que pode ser uma instituição financeira ou um assento. 

Você também pode construir buscas mais avançadas, utilizando os operadores disponíveis na API do Twitter (https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/search-operators)

In [1]:
query = 'coxinha lang:pt'

# 2 - Utilize o seu BEARER_TOKEN para realizar as consultas na API do Twitter.

In [2]:
# Use o seu próprio BEARER_TOKEN
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAEF1gQEAAAAATxx2Gqt0DsT8B%2FMmkbTqLWLxhxI%3DHEvHDuMr50dJLnZIPkfx5pnbW6iE3cf5rKFuAHor3AQEqJtV2L'

# 3 - O código abaixo não precisa ser aberto nem alterado.

In [5]:
#@title Mantenha esse código oculto
from tqdm import tqdm
import json
import csv
import requests
import pandas
import time
from requests.utils import quote
# import emoji
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import nltk
%matplotlib inline
nltk.download('stopwords')
nltk.download('rslp')

def busca_frequencia(palavras):
  palavras = nltk.FreqDist(palavras)
  return palavras

def busca_palavras(frases):
  todas_palavras = []
  for (palavras,sentimento) in frases:
    todas_palavras.extend(palavras)
  return todas_palavras

def aplica_stemmer(texto,lista_stop):
  stemmer = nltk.stem.RSLPStemmer()
  frases_sem_stemming = []
  for (palavras,sentimento) in texto:
    com_stemming = [str(stemmer.stem(p)) for p in palavras if p not in lista_stop]
    frases_sem_stemming.append((com_stemming,sentimento))
  return frases_sem_stemming

def busca_palavras_unicas(frequencia):
  freq = frequencia.keys()
  return freq

def remove_stopwords(texto,lista_stop):
  frases = []
  for (palavras,sentimento) in texto:
    sem_stop = [p for p in palavras.split() if p not in lista_stop]
    frases.append((sem_stop,sentimento))
  return frases

def analizador(tweet,base_treinamento):

  # exemplo_base = pd.DataFrame(base_treinamento)
  # exemplo_base.columns = ['Frase','Sentimento']
  # print('Tamanho da base de treinamento {}.',format(exemplo_base.shape[0]))
  # print(exemplo_base.Sentimento.value_counts())
  # print((exemplo_base.Sentimento.value_counts()/exemplo_base.shape[0])*100)
  # print(exemplo_base.sample(n=3))

  lista_stop = nltk.corpus.stopwords.words('portuguese')
  # lista_stop.append('tão')
  # print(np.transpose(lista_stop))

  base_treinamento = remove_stopwords(base_treinamento,lista_stop)
  # for b in base_treinamento:
  #   print(b)

  frases_com_stem_treinamento = aplica_stemmer(base_treinamento,lista_stop)
  palavras_treinamento = busca_palavras(frases_com_stem_treinamento)
  frequencia_treinamento = busca_frequencia(palavras_treinamento)
  palavras_unicas_treinamento = busca_palavras_unicas(frequencia_treinamento)

  def extrator_palavras(documento):
    doc = set(documento)
    caracteristicas = {}
    for palavras in palavras_unicas_treinamento:
      caracteristicas['%s' %palavras] = (palavras in doc)
    return caracteristicas

  base_completa_treinamento = nltk.classify.apply_features(extrator_palavras,frases_com_stem_treinamento)
  classificador = nltk.NaiveBayesClassifier.train(base_completa_treinamento)
  # print(classificador.labels())

  tweets = [tweet]

  for t in tweets:
    testeStemming = []
    stemmer = nltk.stem.RSLPStemmer()
    for (palavras_treinanemto) in t.split():
      comStem = [p for p in palavras_treinanemto.split()]
      testeStemming.append(str(stemmer.stem(comStem[0])))
    novo = extrator_palavras(testeStemming)

    distribuicao = classificador.prob_classify(novo)
    for classe in distribuicao.samples():
      print('%s: %f' % (classe,distribuicao.prob(classe)))


def create_headers(bearer_token):
    """ Creates headers for requests """
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def create_url(query_field):
    """ Creates a API URL for reuqests """
    url = f'https://api.twitter.com/2/tweets/search/recent?query={query_field}'
    return url

def connect_to_endpoint(url, headers):
    """ Connects to API URL endpoint """
    for i in range(1, 14):
        try:
            response = requests.get(url, headers=headers)
            if response.status_code == 200:
                return response.json()
        except Exception as e:
      
            print("Connection failed, retrying...",i)
            time.sleep(5)

def json_extract(obj, key):
    """Recursively fetch values from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    values = extract(obj, arr, key)
    return values

def main(token):
    stop = False
    tweets = set()
    # for i in tqdm(range(10)):
    for i in range(10):
        print()
        headers = create_headers(BEARER_TOKEN)
        queryHTML = quote(query, safe='')
        
        if token == '':
          url = create_url(queryHTML)
        else:
          url = create_url(queryHTML+'&next_token='+token)  

        time.sleep(3.2) # Sleep for 3 seconds
        try:
            data = connect_to_endpoint(url, headers)
        except Exception as e:
            print("Oh no!!!")
        id = json_extract(data, 'id')
        created_at = json_extract(data, 'id')
        author_id = json_extract(data, 'author_id')
        text = json_extract(data, 'text')

        
        for t in text:
            # print()
            # print(t)
            # t = t.replace('\n',' ')
            # print()
            tweets.add(t)       
        
        try:
            token = json_extract(data, 'next_token')[0]
            with open(token+'.json', 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)
            # print(token)
            break
        except:
            token = ''
            with open('last.json', 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=4, sort_keys=True)
            print("finished")
            stop = True

        if stop:
            break
        # break
    
    
    complete = list(tweets)
    return(complete,token)

    # hashtagList = []
    # for t in complete:
    #         for h in re.findall(r"#(\w+)", t):                
    #             hashtagList.append(h.lower())
    
    # hashtagSet = list(set(hashtagList))
    # hashtagSet.sort()

    # tweetsFile=open('tweets: '+query+'.txt','w')
    # for element in complete:
    #     tweetsFile.write(element)
    #     tweetsFile.write('\n')
    #     tweetsFile.write('\n')
    # tweetsFile.write(str(len(complete)))
    # tweetsFile.close()

    # hashtagFile=open('hashtag: '+query+'.txt','w')
    # for element in hashtagSet:
    #     hashtagFile.write(str(hashtagList.count(element)))
    #     hashtagFile.write(' ')
    #     hashtagFile.write(element)
    #     hashtagFile.write('\n')
    # hashtagFile.write(str(len(hashtagSet)))
    # hashtagFile.close()
    


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


# 4 - Altere o código abaixo e crie o seu próprio classificador. No exemplo abaixo, estamos classificando os Tweets com a palavra banco como instituição, assento ou indefinido.

In [3]:
def classifica(tweets):
  base_treinamento = []
  for t in tweets:
    print(t)
    tipo = input("Comida(1), Partido Politico(2), Indefinido(0)?")
    if tipo == '1':
      base_treinamento.append((t,'comida'))
    elif tipo == '2':
      base_treinamento.append((t,'partidoPolitico'))
    else:
      base_treinamento.append((t,'indefinido'))
  return base_treinamento

# 5 - A função principal executa as seguintes tarefas
1. Utiliza a chave de busca para coletar tweets
1. Utiliza os 10 primeiros tweets coletados para inicializar a base de treinamento
1. Continua as coletas e executa a classificação dos próximos Tweets
1. Caso a análise não esteja boa, você pode melhora-la.



In [ ]:
if __name__ == '__main__':
  
  # Faz uma coleta inicial de Tweets
  token = ''
  tweets,token = main(token)  

  base_treinamento = []
  # Utiliza os primeiros tweets para alimentar a base de treinamento
  base_treinamento = classifica(tweets)
  print(base_treinamento)

  # Continua buscando mais Tweets
  while(token != ''):
    # Utiliza token para continuar buscando Tweets do último ponto de retorno
    tweets,token = main(token) 
    for t in tweets:
      print(t)
      analizador(t,base_treinamento)
      print()
    resposta = input('A analise está boa? s/n')
    if resposta == 'n':
      print('Vamos classificar melhor esses tweets...')
      for i in classifica(tweets):
        base_treinamento.append(i)


# 6 - Sua tarefa: defina a sua chave de busca e altere o seu código do classificador. Realize seus testes para se certificar que o classificador funciona corretamente. Inclua quantas classes forem necessárias. Ao terminar, faça o download deste Colab e envie através do AVA.